In [ ]:
database_path = r"D:\iliapl\topic_modeling\data\databases\53k_individual_hcps_70_percent_confidence_tweets.db"
output_path = r'D:\iliapl\topic_modeling\data\output_data\twitter_location_analysis\author_location_data_53k_authors'

In [ ]:
import sqlite3 as sql
import pandas as pd

con = sql.connect(database_path)
authors_df = pd.read_sql('SELECT * FROM authors', con)

In [ ]:
authors_df = authors_df[['name', 'author_osn_id', 'location']]

In [ ]:
from geopy.geocoders import Bing
from geopy.exc import GeocoderTimedOut, GeocoderQueryError, GeocoderQuotaExceeded, GeocoderServiceError

geolocator = Bing(api_key="AgSzclXa0ydRjeopKMp_qtSEob_A6_LHG8-bq2RMnGIadjGRPRUKANZiKlKdLw4I")

In [ ]:
premade_parsed_location_dict_path = r'D:\iliapl\topic_modeling\data\output_data\twitter_location_analysis\POI_Followers_13-06-20\country_name_dict.json'

In [ ]:
import json

bad_country_names = set()
bad_state_names = set()

state_name_dict = {}

with open(premade_parsed_location_dict_path, 'r') as f:
    parsed_location_dict = json.load(f)

In [ ]:
from geopy.geocoders import Bing
from geopy.exc import GeocoderTimedOut, GeocoderQueryError, GeocoderQuotaExceeded, GeocoderServiceError
import pickle
import time

def get_country_name_by_bing_api(parsed_location, country_type):
    if country_type != 'country' and country_type != 'state':
        print('Country type must be either country state')
        return "", True
    if not parsed_location.strip():
        return "", True
    if parsed_location in bad_country_names:
        return "", True    
    if country_type == 'state' and parsed_location in bad_state_names:
        return "", True
    if country_type == 'state':
        for state_exception in state_exceptions:
            if state_exception in parsed_location.lower():
                return state_exceptions[state_exception], True
    if country_type == 'country' and parsed_location in parsed_location_dict:
        return parsed_location_dict[parsed_location], True
    if country_type == 'state' and parsed_location in state_name_dict:
        return state_name_dict[parsed_location], True
    
    try:
        location = geolocator.geocode(parsed_location)
        if location is None: # fail
            bad_country_names.add(parsed_location)
            return "", False
        if 'address' not in location.raw:
            bad_country_names.add(parsed_location)
            return "", False
        if 'countryRegion' not in location.raw['address']:
            bad_country_names.add(parsed_location)
            return "", False
        country_name = location.raw['address']['countryRegion']
        parsed_location_dict[parsed_location] = country_name
        if country_name == 'United States':
            if 'adminDistrict' not in location.raw['address']:
                bad_state_names.add(parsed_location)
                if country_type == 'state':
                    return "", False
            else:
                state = location.raw['address']['adminDistrict']
                state_name_dict[parsed_location] = state
                if country_type == 'state':
                    return state, False
        else:
            if country_type == 'state':
                return "", False
        if country_type == 'country':
            return country_name, False
        else:
            bad_country_names.add(parsed_location)
            return "", False

    except GeocoderTimedOut as e:
        print(e)
        time.sleep(2)
        try:
            location = geolocator.geocode(parsed_location)
        except GeocoderTimedOut as e2:
            success = False
            while not success:
                time.sleep(5)
                try:
                    location = geolocator.geocode(parsed_location)
                    success = True
                except GeocoderTimedOut as e3:
                    pass
        if location is None: # fail
            bad_country_names.add(parsed_location)
            return "", False
        if 'address' not in location.raw:
            bad_country_names.add(parsed_location)
            return "", False
        if 'countryRegion' not in location.raw['address']:
            bad_country_names.add(parsed_location)
            return "", False
        country_name = location.raw['address']['countryRegion']
        parsed_location_dict[parsed_location] = country_name
        if country_name == 'United States':
            if 'adminDistrict' not in location.raw['address']:
                bad_state_names.add(parsed_location)
                print('ATTENTION: {} added to bad states set'.format(parsed_location))
                if country_type == 'state':
                    return "", False
            else:
                state = location.raw['address']['adminDistrict']
                state_name_dict[parsed_location] = state
                if country_type == 'state':
                    return state, False
        else:
            if country_type == 'state':
                return "", False
        if country_type == 'country':
            return country_name, False
        else:
            bad_country_names.add(parsed_location)
            return "", False
    except GeocoderQueryError as e:
        bad_country_names.add(parsed_location)
        print('Query error! query: {}'.format(parsed_location))
        return "Bad_Request", False

    except GeocoderQuotaExceeded as e:
        print('Quota exceeded!')
        time.sleep(1.5)
        return "Bad_Request", False
    
    except GeocoderServiceError as e:
        print('Service error!')
        time.sleep(1)
        return "Bad_Request", False
        

In [ ]:
twitter_author_initial_location_dict = {}

for entry in authors_df.itertuples():
    twitter_author_initial_location_dict[(entry.name, str(entry.author_osn_id))] = entry.location

In [ ]:
author_country_dict = {}

In [ ]:
import time

bad_requests = 0
num_requests_total = 0
start_time = time.time()
for i, (author_tup, location) in enumerate(twitter_author_initial_location_dict.items(), 1):
    
    author_osn_id = author_tup[1]
    
    country_name, is_cached = get_country_name_by_bing_api(location, 'country')
    if not is_cached:
        num_requests_total += 1
        if num_requests_total % 500 == 0:
            with open('{}/parsed_location_dict.json'.format(output_path), 'w') as handle:
                json.dump(parsed_location_dict, handle)
            #with open('{}/state_name_dict.json'.format(output_path), 'w') as handle:
            #    json.dump(state_name_dict, handle)
            with open('{}/bad_country_names.json'.format(output_path), 'w') as handle:
                json.dump(list(bad_country_names), handle)
            with open('{}/bad_state_names.json'.format(output_path), 'w') as handle:
                json.dump(list(bad_state_names), handle)
            with open('{}/author_country_dict.json'.format(output_path), 'w') as handle:
                json.dump(author_country_dict, handle)

            print("Dictionary was saved! ")

    if country_name == 'Bad_Request':
        bad_requests += 1
    else:
        if country_name:
            author_country_dict[author_osn_id] = country_name
    if i % 10000 == 0:
        print('Finished {} authors. So far {} requests total. {} seconds total.'.format(i, num_requests_total, time.time() - start_time))
        
print('Finished {} authors. {} bad requests.'.format(len(author_country_dict), bad_requests))

with open('{}/parsed_location_dict.json'.format(output_path), 'w') as handle:
    json.dump(parsed_location_dict, handle)
with open('{}/state_name_dict.json'.format(output_path), 'w') as handle:
    json.dump(state_name_dict, handle)
with open('{}/bad_country_names.json'.format(output_path), 'w') as handle:
    json.dump(list(bad_country_names), handle)
with open('{}/bad_state_names.json'.format(output_path), 'w') as handle:
    json.dump(list(bad_state_names), handle)
    
with open('{}/author_country_dict.json'.format(output_path), 'w') as handle:
    json.dump(author_country_dict, handle)

print("Dictionary was saved!")

In [ ]:
author_country_dict